In [48]:
import pandas as pd
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, \
    pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

# Görev 1: Veriyi Hazırlama ve Analiz Etme

* Adım 1: ab_testing_data.xlsx adlı kontrol ve test grubu verilerinden oluşan veri setini okutunuz. Kontrol ve test grubu verilerini ayrı
değişkenlere atayınız.

In [49]:
df_kontrol=pd.read_excel("ab_testing.xlsx",sheet_name="Control Group")

In [50]:
df_kontrol.head()

,Impression,Click,Purchase,Earning
0,82529.459271,6090.077317,665.211255,2311.277143
1,98050.451926,3382.861786,315.084895,1742.806855
2,82696.023549,4167.965750,458.083738,1797.827447
3,109914.400398,4910.882240,487.090773,1696.229178
4,108457.762630,5987.655811,441.034050,1543.720179


In [51]:
df_test=pd.read_excel("ab_testing.xlsx",sheet_name="Test Group")

In [52]:
df_test.head()

,Impression,Click,Purchase,Earning
0,120103.503796,3216.547958,702.160346,1939.611243
1,134775.943363,3635.082422,834.054286,2929.405820
2,107806.620788,3057.143560,422.934258,2526.244877
3,116445.275526,4650.473911,429.033535,2281.428574
4,145082.516838,5201.387724,749.860442,2781.697521


* Adım 2: Kontrol ve test grubu verilerini analiz ediniz.

In [53]:
df_test.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.0,120512.411758,18807.448712,79033.834921,112691.970770,119291.300775,132050.578933,158605.920483
Click,40.0,3967.549761,923.095073,1836.629861,3376.819024,3931.359804,4660.497911,6019.695079
Purchase,40.0,582.106097,161.152513,311.629515,444.626828,551.355732,699.862360,889.910460
Earning,40.0,2514.890733,282.730852,1939.611243,2280.537426,2544.666107,2761.545405,3171.489708


In [54]:
df_kontrol.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.0,101711.449068,20302.157862,45475.942965,85726.690349,99790.701078,115212.816543,147539.336329
Click,40.0,5100.657373,1329.985498,2189.753157,4124.304129,5001.220602,5923.803596,7959.125069
Purchase,40.0,550.894059,134.108201,267.028943,470.095533,531.206307,637.957088,801.795020
Earning,40.0,1908.568300,302.917783,1253.989525,1685.847205,1975.160522,2119.802784,2497.295218


In [55]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Impression  40 non-null     float64
 1   Click       40 non-null     float64
 2   Purchase    40 non-null     float64
 3   Earning     40 non-null     float64
dtypes: float64(4)
memory usage: 1.4 KB


In [56]:
df_kontrol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Impression  40 non-null     float64
 1   Click       40 non-null     float64
 2   Purchase    40 non-null     float64
 3   Earning     40 non-null     float64
dtypes: float64(4)
memory usage: 1.4 KB


* Adım 3: Analiz işleminden sonra concat metodunu kullanarak kontrol ve test grubu verilerini birleştiriniz.

In [57]:
# İki farklı veri setini tek bir veri setinde birleştireceğimiz için gözlemlerin hangi veriden geldiğini
# belirtmek amacıyla yeni bir sütun oluşturduk.
df_kontrol['source'] = 'max'
df_test['source'] = 'average'

In [58]:
df=pd.concat([df_kontrol,df_test])

In [59]:
df.head()

,Impression,Click,Purchase,Earning,source
0,82529.459271,6090.077317,665.211255,2311.277143,max
1,98050.451926,3382.861786,315.084895,1742.806855,max
2,82696.023549,4167.965750,458.083738,1797.827447,max
3,109914.400398,4910.882240,487.090773,1696.229178,max
4,108457.762630,5987.655811,441.034050,1543.720179,max


In [71]:
df.tail()

,Impression,Click,Purchase,Earning,source
35,79234.911929,6002.213585,382.047116,2277.863984,average
36,130702.239410,3626.320072,449.824592,2530.841327,average
37,116481.873365,4702.782468,472.453725,2597.917632,average
38,79033.834921,4495.428177,425.359102,2595.857880,average
39,102257.454089,4800.068321,521.310729,2967.518390,average


In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80 entries, 0 to 39
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Impression  80 non-null     float64
 1   Click       80 non-null     float64
 2   Purchase    80 non-null     float64
 3   Earning     80 non-null     float64
 4   source      80 non-null     object 
dtypes: float64(4), object(1)
memory usage: 3.8+ KB


# Görev 2: A/B Testinin Hipotezinin Tanımlanması

* Adım 1: Hipotezi tanımlayınız.
* H0 : M1 = M2
* H1 : M1!= M2

* Adım 2: Kontrol ve test grubu için purchase (kazanç) ortalamalarını analiz ediniz.

In [61]:
df_kontrol["Purchase"].mean()

550.8940587702316

In [62]:
df.loc[df["source"]=="max","Purchase"].mean()

550.8940587702316

In [63]:
df_test["Purchase"].mean()

582.1060966484677

In [64]:
df.loc[df["source"]=="average","Purchase"].mean()

582.1060966484677

In [65]:
df["Purchase"].mean()

566.5000777093495

# Görev 3: Hipotez Testinin Gerçekleştirilmesi

* Adım 1: Hipotez testi yapılmadan önce varsayım kontrollerini yapınız.
Bunlar Normallik Varsayımı ve Varyans Homojenliğidir. Kontrol ve test grubunun normallik varsayımına uyup uymadığını Purchase değişkeni
üzerinden ayrı ayrı test ediniz.

* _Normallik Varsayımı_ :
* H0: Normal dağılım varsayımı sağlanmaktadır.
* H1: Normal dağılım varsayımı sağlanmamaktadır.
* p < 0.05 H0 RED , p > 0.05 H0 REDDEDİLEMEZ
* Test sonucuna göre normallik varsayımı kontrol ve test grupları için sağlanıyor mu ? Elde edilen p-value değerlerini yorumlayınız.

* _Varyans Homojenliği_ :
* H0: Varyanslar homojendir.
* H1: Varyanslar homojen Değildir.
* p < 0.05 H0 RED , p > 0.05 H0 REDDEDİLEMEZ
* Kontrol ve test grubu için varyans homojenliğinin sağlanıp sağlanmadığını Purchase değişkeni üzerinden test ediniz.
* Test sonucuna göre normallik varsayımı sağlanıyor mu? Elde edilen p-value değerlerini yorumlayınız.
### Adım 2: Normallik Varsayımı ve Varyans Homojenliği sonuçlarına göre uygun testi seçiniz.

In [66]:
test_stat, pvalue = shapiro(df.loc[df['source'] == 'max', 'Purchase'])

In [67]:
pvalue

0.5891125202178955

In [68]:
test_stat, pvalue = shapiro(df.loc[df['source'] == 'average', 'Purchase'])
pvalue

0.15413342416286469

* Kontrol grubu için p değeri 0.5891'dir.  p > 0.05 olduğu için HO reddedilemez. Dolayısıyla normallik varsayımı
 karşılanmış olur.

* Test grubu için p değeri 0.1541'dir.  p > 0.05 olduğu için HO reddedilemez. Dolayısıyla normallik varsayımı
 karşılanmış olur.

In [69]:
test_stat, pvalue = levene(df.loc[df['source'] == 'max', 'Purchase'],
                           df.loc[df['source'] == 'average', 'Purchase'])
pvalue

0.10828588271874791

* p değeri 0.1083'tür. Dolayısıyla HO reddedilemez . Varyanslar homojendir

### Adım 3: Test sonucunda elde edilen p_value değerini göz önünde bulundurarak kontrol ve test grubu satın alma ortalamaları arasında istatistiki olarak anlamlı bir fark olup olmadığını yorumlayınız.

In [70]:
test_stat, pvalue = ttest_ind(df.loc[df['source'] == 'max', 'Purchase'],
                              df.loc[df['source'] == 'average', 'Purchase'],
                              equal_var=True)
pvalue

0.34932579202108416

* P değeri 0.3493'tür. p > 0.05 olduğu için H0 reddedilemez.
* H0 hipotezi reddedilemediği için iki grup arasında istatistiki olarak anlamlı bir farklılık bulunmamaktadır.
* Ortalamalar arasındaki farklar tesadüftür.

# Görev 4: Sonuçların Analizi
* Normallik ve homojenlik varsayımlarının yaptığımız testler sonucunda sağlandığı görülmektedir.
* Bu iki varsayımın karşılandığı koşullarda uygulayabileceğimiz bağımsız iki örneklem t testi ( parametrik test )
 bu çalışma için uygun görülmüştür.

* İşletmenin başarı ölçütü olarak belirlediği Purchase değişkeninde istatistiki olarak anlamlı bir gelişme kaydedilmediği görülmektedir. Bir süre daha bu değişken özelinde AB testleri yapılabilir ve gelişme kaydedilip kaydedilmediği gözlemlenmeye devam edilebilir.Mevcut koşullarda yeni teklif türünün daha avantajlı olduğunu bilimsel açıdan söyleyemeyiz